# RetroChem

Welcome to the RetroChem documentation notebook—your interactive guide to our retrosynthesis toolkit! In this notebook, we’ll walk you through our design process, highlight key challenges we encountered, and showcase how RetroChem can streamline organic synthesis planning.

Below is the table of contents to help you navigate the main sections:

1. **Introduction**  
2. **Materials & Methods**  
3. **Results** 
4. **Discussion**  

# Introduction

Organic chemistry is the beating heart of any chemistry curriculum, an immense tapestry of functional groups, transformations, and reaction pathways. With thousands of named reactions and countless mechanistic subtleties to master, the sheer volume of information can overwhelm even the most dedicated student. What better way to tame this complexity than with the power of code?

**RetroChem** is our answer: a web-based retrosynthesis assistant that translates familiar IUPAC names or drawn structures into SMILES, and then walks you backward through our hand-curated library of key organic transformations, showing each disconnection step along with the reagents and conditions required to carry it out. By combining cheminformatics tools (RDKit, SMARTS pattern matching) with interactive web widgets, RetroChem aims to kick-start the digital revolution in organic synthesis: helping you explore, learn, and visualize reaction pathways at the click of a button.  

Welcome to the future of organic chemistry education. Let’s turn that mountain of reactions into a streamlined, code-driven discovery journey!  


#Materials and Methods 

# No code without imports! 

In [ ]:
import streamlit as st
import pandas as pd
import os
from streamlit_ketcher import st_ketcher  # type: ignore
from rdkit import Chem
from rdkit.Chem.Draw import MolToImage

from retrochem.functions import name_to_smiles, canonicalize_smiles
import retrochem.reaction_database as rd

# Let's get started ! 

First thing first, the user has to select a database in which he might find the reaction that will lead to his input molecule. There are several databases available. Let's select for example the EPFL students database. 

In [ ]:
dbs = list(rd.REACTION_DATABASES.keys())
if dbs:
    cols = st.columns(len(dbs))
    for i, d in enumerate(dbs):
        with cols[i]:
            st.button(d, on_click=choose_database, args=(d,))
else:
    st.info("No .db files found.")


# Inputing your output molecule 

First things first: we need to let the user provide a target molecule—either by typing its IUPAC/common name or by drawing its structure—and then convert that input into a SMILES string using our name_to_smiles or structure_to_smiles helper functions.
For the name_to_smiles function, it reaches out to the **NCI’s Chemical Identifier Resolver (Cactus)** to look up any common or IUPAC name you type in and return the corresponding SMILES string. 

# Usage Example

Let' say the user inputs 1,1 dietoxyethane, whether as a name or a strcuture: these two functions should convert to smiles: 

In [ ]:
if mode == "Name":
    name = st.text_input("Molecule name")
    if name:
        try:
            smiles_input = name_to_smiles(name)
            st.write("SMILES:", smiles_input)
            mol0 = Chem.MolFromSmiles(smiles_input)
            if mol0:
                st.image(MolToImage(mol0, size=(200, 200)))
        except Exception as e:
            st.error(f"❌ Name→SMILES failed: {e}")
else:
    drawn = st_ketcher("", key="draw_input")
    if drawn:
        smiles_input = drawn
        st.write("SMILES:", smiles_input)
        mol0 = Chem.MolFromSmiles(smiles_input)
        if mol0:
            st.image(MolToImage(mol0, size=(200, 200)))

# Getting the possible reactants 

Now things start to get serious !! 

We're going to walk you through how the retrosynthesis happens. 